In [1]:
from __future__ import division
from __future__ import print_function
from sklearn import metrics
import random
import time
import sys
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import dgl
import dgl.function as fn
from dgl import DGLGraph
import numpy as np

from utils.utils import *
from models.gcn import GCN
from models.mlp import MLP

Using backend: pytorch


In [2]:
# Loading graph
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus()
features = sp.identity(features.shape[0])
features = preprocess_features(features)

In [3]:
features.shape

(23551, 23551)

In [5]:
# Loading image features
train_embeddings, test_embeddings = get_image_embeddings()
training_embeddings = torch.tensor(train_embeddings).reshape(train_size,512)
test_embeddings = torch.tensor(test_embeddings).reshape(test_size,512)


# Getting complete features for all nodes
word_nodes = features.shape[0] - train_size - test_size

# Since we don't have image embeddings for words, we will use zeros
image_embeddings_words = torch.zeros(word_nodes,512)

all_image_features = torch.cat((training_embeddings, image_embeddings_words, test_embeddings), 0)



In [7]:
all_image_features

tensor([[ 0.2074, -0.2249, -0.0530,  ...,  0.0562,  0.0360,  0.0046],
        [ 0.3254,  0.1113, -0.0094,  ..., -0.0314,  0.4260, -0.3494],
        [ 0.1094, -0.4641, -0.1338,  ...,  0.0568,  0.0364, -0.1023],
        ...,
        [-0.2140, -0.1322,  0.0471,  ...,  0.2145,  0.1888, -0.2120],
        [-0.0359,  0.0592,  0.2881,  ...,  0.5137, -0.0331,  0.1102],
        [ 0.0183,  0.2300, -0.2666,  ...,  0.0280,  0.0512,  0.0186]])

In [4]:
def pre_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj + sp.eye(adj.shape[0]))
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

adjdense = torch.from_numpy(pre_adj(adj).A.astype(np.float32))

In [5]:
CUDA = True
def construct_graph(adjacency):
    g = DGLGraph()
    adj = pre_adj(adjacency)
    g.add_nodes(adj.shape[0])
    g.add_edges(adj.row,adj.col)
    adjdense = adj.A
    adjd = np.ones((adj.shape[0]))
    for i in range(adj.shape[0]):
        adjd[i] = adjd[i] * np.sum(adjdense[i,:])
    weight = torch.from_numpy(adj.data.astype(np.float32))
    g.ndata['d'] = torch.from_numpy(adjd.astype(np.float32))
    g.edata['w'] = weight

    if CUDA:
        g = g.to(torch.device('cuda:0'))
    
    return g

In [6]:
class SimpleConv(nn.Module):
    def __init__(self,g,in_feats,out_feats,activation,feat_drop=True):
        super(SimpleConv, self).__init__()
        self.graph = g
        self.activation = activation
        #self.reset_parameters()
        setattr(self, 'W', nn.Parameter(torch.randn(in_feats,out_feats)))
        #self.b = nn.Parameter(torch.zeros(1, out_feats))
        #self.linear = nn.Linear(in_feats,out_feats)
        self.feat_drop = feat_drop
    
    # def reset_parameters(self):
    #     gain = nn.init.calculate_gain('relu')
    #     nn.init.xavier_uniform_(self.linear.weight,gain=gain)
    
    def forward(self, feat):
        g = self.graph.local_var()
        g.ndata['h'] = feat.mm(getattr(self, 'W'))
        g.update_all(fn.src_mul_edge(src='h', edge='w', out='m'), fn.sum(msg='m',out='h'))
        rst = g.ndata['h']
        #rst = self.linear(rst)
        rst = self.activation(rst)
        return rst

In [15]:

class Classifer(nn.Module):
    def __init__(self,g,input_dim,num_classes,conv):
        super(Classifer, self).__init__()
        self.data_graph = g
        self.GCN = conv
        self.gcn1 = self.GCN(g,input_dim, 300, F.relu)
        self.gcn2 = self.GCN(g, 300, 200, F.relu)
        self.gcn3 = self.GCN(g, 200, num_classes, F.relu)

    
    def forward(self, features):
        x = self.gcn1(features)

        # To Do: Fuse the text embedding with image embedding 
        self.embedding = x
        # x = torch.cat(x,g.ndata['x'])
        # x = torch.cat((self.embedding,g.ndata['x']),dim=1)
        x = self.gcn2(x)
        x = self.gcn3(x)
        
        return x




class ClassiferFusion(nn.Module):
    def __init__(self,g,input_dim,num_classes,conv):
        super(Classifer, self).__init__()
        self.data_graph = g
        self.GCN = conv
        self.gcn1 = self.GCN(g,input_dim, 300, F.relu)
        self.gcn2 = self.GCN(g, 300, 200, F.relu)
        self.gcn3 = self.GCN(g, 100, num_classes, F.relu)

    
    def forward(self, features):
        x = self.gcn1(features)

        # To Do: Fuse the text embedding with image embedding 
        self.embedding = x
        # x = torch.cat(x,g.ndata['x'])
        x = torch.cat((self.embedding,g.ndata['x']),dim=1)
        x = self.gcn2(x)
        x = self.gcn3(x)
        
        return x

In [8]:
g = construct_graph(adj)


/home/bis/Projects/venv3.6/lib/python3.6/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [16]:
model = Classifer(g,input_dim=features.shape[0], num_classes=y_train.shape[1],conv=SimpleConv)

In [10]:
# Define placeholders
t_features = torch.from_numpy(features.astype(np.float32))
t_y_train = torch.from_numpy(y_train)
t_y_val = torch.from_numpy(y_val)
t_y_test = torch.from_numpy(y_test)
t_train_mask = torch.from_numpy(train_mask.astype(np.float32))
tm_train_mask = torch.transpose(torch.unsqueeze(t_train_mask, 0), 1, 0).repeat(1, y_train.shape[1])
support = [preprocess_adj(adj)]
num_supports = 1
t_support = []
for i in range(len(support)):
    t_support.append(torch.Tensor(support[i]))

In [17]:
t_features = t_features.cuda()
t_y_train = t_y_train.cuda()
#t_y_val = t_y_val.cuda()
#t_y_test = t_y_test.cuda()
t_train_mask = t_train_mask.cuda()
tm_train_mask = tm_train_mask.cuda()
# for i in range(len(support)):
#     t_support = [t.cuda() for t in t_support if True]
model = model.cuda()

In [18]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [19]:
def evaluate(features, labels, mask):
    t_test = time.time()
    # feed_dict_val = construct_feed_dict(
    #     features, support, labels, mask, placeholders)
    # outs_val = sess.run([model.loss, model.accuracy, model.pred, model.labels], feed_dict=feed_dict_val)
    model.eval()
    with torch.no_grad():
        logits = model(features).cpu()
        t_mask = torch.from_numpy(np.array(mask*1., dtype=np.float32))
        tm_mask = torch.transpose(torch.unsqueeze(t_mask, 0), 1, 0).repeat(1, labels.shape[1])
        loss = criterion(logits * tm_mask, torch.max(labels, 1)[1])
        pred = torch.max(logits, 1)[1]
        acc = ((pred == torch.max(labels, 1)[1]).float() * t_mask).sum().item() / t_mask.sum().item()
        
    return loss.numpy(), acc, pred.numpy(), labels.numpy(), (time.time() - t_test)

val_losses = []

In [20]:
# Train model
epochs = 50
for epoch in range(epochs):

    t = time.time()
    
    # Forward pass
    logits = model(t_features)
    loss = criterion(logits * tm_train_mask, torch.max(t_y_train, 1)[1])    
    acc = ((torch.max(logits, 1)[1] == torch.max(t_y_train, 1)[1]).float() * t_train_mask).sum().item() / t_train_mask.sum().item()
        
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation
    val_loss, val_acc, pred, labels, duration = evaluate(t_features, t_y_val, val_mask)
    val_losses.append(val_loss)

    print_log("Epoch: {:.0f}, train_loss= {:.5f}, train_acc= {:.5f}, val_loss= {:.5f}, val_acc= {:.5f}, time= {:.5f}"\
                .format(epoch + 1, loss, acc, val_loss, val_acc, time.time() - t))

    # if epoch > 5 and val_losses[-1] > np.mean(val_losses[-(5+1):-1]):
    #     print_log("Early stopping...")
    #     break


print_log("Optimization Finished!")

[2022/4/4 18:56:58] Epoch: 1, train_loss= 2.65577, train_acc= 0.37435, val_loss= 1.67588, val_acc= 0.51800, time= 0.67278
[2022/4/4 18:56:58] Epoch: 2, train_loss= 2.05875, train_acc= 0.38706, val_loss= 1.64236, val_acc= 0.53200, time= 0.62460
[2022/4/4 18:56:59] Epoch: 3, train_loss= 1.54058, train_acc= 0.41576, val_loss= 1.61902, val_acc= 0.52200, time= 0.62247
[2022/4/4 18:57:00] Epoch: 4, train_loss= 1.20177, train_acc= 0.48835, val_loss= 1.60657, val_acc= 0.52200, time= 0.62568
[2022/4/4 18:57:00] Epoch: 5, train_loss= 1.13552, train_acc= 0.56788, val_loss= 1.60399, val_acc= 0.51200, time= 0.62609
[2022/4/4 18:57:01] Epoch: 6, train_loss= 1.23361, train_acc= 0.60165, val_loss= 1.60470, val_acc= 0.50600, time= 0.63442
[2022/4/4 18:57:01] Epoch: 7, train_loss= 1.34360, train_acc= 0.61765, val_loss= 1.60545, val_acc= 0.50400, time= 0.64002
[2022/4/4 18:57:02] Epoch: 8, train_loss= 1.41096, train_acc= 0.62553, val_loss= 1.60470, val_acc= 0.51000, time= 0.63683
[2022/4/4 18:57:03] Epoc

In [21]:
# Testing
test_loss, test_acc, pred, labels, test_duration = evaluate(t_features, t_y_test, test_mask)
print_log("Test set results: \n\t loss= {:.5f}, accuracy= {:.5f}, time= {:.5f}".format(test_loss, test_acc, test_duration))

test_pred = []
test_labels = []
for i in range(len(test_mask)):
    if test_mask[i]:
        test_pred.append(pred[i])
        test_labels.append(np.argmax(labels[i]))


print_log("Test Precision, Recall and F1-Score...")
print_log(metrics.classification_report(test_labels, test_pred, digits=4))
print_log("Macro average Test Precision, Recall and F1-Score...")
print_log(metrics.precision_recall_fscore_support(test_labels, test_pred, average='macro'))
print_log("Micro average Test Precision, Recall and F1-Score...")
print_log(metrics.precision_recall_fscore_support(test_labels, test_pred, average='micro'))

[2022/4/4 18:57:31] Test set results: 
[2022/4/4 18:57:31] 	 loss= 0.79779, accuracy= 0.52000, time= 0.26450
[2022/4/4 18:57:31] Test Precision, Recall and F1-Score...
[2022/4/4 18:57:31]               precision    recall  f1-score   support
[2022/4/4 18:57:31] 
[2022/4/4 18:57:31]            0     0.5254    0.6078    0.5636       510
[2022/4/4 18:57:31]            1     0.5122    0.4286    0.4667       490
[2022/4/4 18:57:31] 
[2022/4/4 18:57:31]     accuracy                         0.5200      1000
[2022/4/4 18:57:31]    macro avg     0.5188    0.5182    0.5152      1000
[2022/4/4 18:57:31] weighted avg     0.5189    0.5200    0.5161      1000
[2022/4/4 18:57:31] 
[2022/4/4 18:57:31] Macro average Test Precision, Recall and F1-Score...
[2022/4/4 18:57:31] (0.5188094253823894, 0.5182072829131652, 0.5151515151515151, None)
[2022/4/4 18:57:31] Micro average Test Precision, Recall and F1-Score...
[2022/4/4 18:57:31] (0.52, 0.52, 0.52, None)
